In [ ]:
import sys

sys.path.append('C:/Users/wanglab/Documents/Whisker-Analysis/out/build/MSVC/Release')
#sys.path.append('C:/Users/wanglab/Documents/Whisker-Analysis/out/build/Clang/Release')
#sys.path.append('../out/build/MSVC/Release')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import whiskertracker

# Create an instance of WhiskerTracker
wt = whiskertracker.WhiskerTracker()

wt.setWhiskerPad(582, 267)

img = cv2.imread('C:/Users/wanglab/Desktop/img0011018.png')

if len(img.shape) > 2:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Flatten the image and convert it to a list of uint8_t
image_data = img.flatten().tolist()

wt.trace(image_data,480,640)

In [ ]:
dilation_size = 13
data_path = "C:/Users/wanglab/Data/Piezo/NTNG-1-Piezo/053124/4588/1/"
mask_path = data_path + "mask.png"
video_path = data_path + "output.mp4"
num_whiskers = 5
pad_pos = [566, 275]

wt.setWhiskerPad(pad_pos[0], pad_pos[1])


In [ ]:

face_mask = cv2.imread(mask_path)

gray_mask = cv2.cvtColor(face_mask, cv2.COLOR_BGR2GRAY)

# Step 3: Apply threshold
_, thresh = cv2.threshold(gray_mask, 50, 255, cv2.THRESH_BINARY)

# Grow mask to appropriate size
element = np.ones((dilation_size, dilation_size), np.uint8)
cv2.bitwise_not(thresh, thresh)
cv2.dilate(thresh, element, thresh, iterations=1)
cv2.bitwise_not(thresh, thresh)

# Step 4: Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty image for drawing contours
contour_img = 255 * np.ones_like(face_mask)

# Step 5: Draw contours
cv2.drawContours(contour_img, contours, -1, (0, 0, 0), 3)  # Adjust thickness here

gray_mask = cv2.cvtColor(contour_img, cv2.COLOR_BGR2GRAY)

threshold = 50

black_pixels = [(x, y) for y in range(gray_mask.shape[0]) for x in range(gray_mask.shape[1]) if gray_mask[y, x] < threshold]

wt.setFaceMask(black_pixels)

In [ ]:
#Clang fast
%timeit wt.trace(image_data,480,640)

In [ ]:
image_data = img.flatten().tolist()
wt.trace(image_data,480,640)

In [ ]:
# MSVC fast
%timeit wt.trace(image_data,480,640)

In [ ]:
for i in range(0,len(w0)):
    ww = np.reshape(w0[i],(-1,2))
    plt.plot(ww[:,0],ww[:,1])
plt.xlim(0,640)
plt.ylim(0,480)

In [ ]:
vidcap = cv2.VideoCapture(video_path)

num_frames = get_total_frames(vidcap)

iterable = create_progress_bar(num_frames)

whiskers_per_frame = []
frames = []

output_frames = []

for i, frame in iterable:
    success, img = vidcap.read()

    if success:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        image_data = img.flatten().tolist()

        wt.trace(image_data,480,640)

        w0 = wt.getWhiskers()

        frames.append(i)

        whiskers_per_frame.append([])
        for j in range(0,len(w0)):
            ww = np.reshape(w0[j],(-1,2))
            whiskers_per_frame[-1].append(ww)
    else:
        print(f"Failed to read frame {i}")


vidcap.release()


In [ ]:
frame_to_plot = 8
for i in range(0,len(whiskers_per_frame[frame_to_plot])):
    ww = whiskers_per_frame[frame_to_plot][i]
    plt.plot(ww[:,0],ww[:,1])

plt.xlim(0,640)
plt.ylim(0,480)

In [ ]:
for w_id in range(0,num_whiskers):
    with h5py.File(data_path + f"whisker_{w_id}.h5", "w") as f:
        frames = np.array(frames, dtype=np.int64)
        f.create_dataset("frames", data=frames, dtype=np.int64)

        dt_float = h5py.vlen_dtype(np.dtype("float32"))

        x = np.empty(len(frames),dtype=object)
        y = np.empty(len(frames),dtype=object)

        for i, whiskers in enumerate(whiskers_per_frame):

            if len(whiskers) <= w_id:
                x[i] = np.array([],dtype=np.float32)
                y[i] = np.array([],dtype=np.float32)
                continue

            w = whiskers[w_id]

            x[i] = w[:,1]
            y[i] = w[:,0]

        f.create_dataset("x", data=x, dtype=dt_float)
        f.create_dataset("y", data=y, dtype=dt_float)


In [ ]:
from tqdm import tqdm
import h5py

def create_progress_bar(num_frames, label="Train"):
    """
    Creates a progress bar that increments with each frame processed

    Parameters
    ----------
    num_frames: int

    Returns
    -------

    """
    iterable = enumerate(range(0, num_frames))
    progress = tqdm(
        iterable, desc=label, total=num_frames, ascii=True, leave=True, position=0
    )
    iterable = progress
    return iterable


def get_total_frames(vidcap):
    num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total number of frames in video {num_frames}")
    return num_frames